In [1]:
from geoband.API import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import json
from pandas import json_normalize
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from folium import Choropleth
import matplotlib
from datetime import datetime 
# GetCompasData로 주피터 노트북 경로에 파일 저장
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
matplotlib.rcParams['font.family']="Malgun Gothic"

In [2]:
# 원래 데이터
suwon_112 = pd.read_csv('1.수원시_112신고이력(격자_관할경계매핑).csv')
others_112 = pd.read_csv('3.3개시도_112신고이력(관할경계매핑).csv')
others_alcohol = gpd.read_file('8.4개시도_유흥업소현황.geojson')
others_police_box_area = gpd.read_file('12.4개시도_치안안전센터_관할경계.geojson')
others_security_lamp = pd.read_csv('22.4개시도_보안등현황.csv')
others_security_bell = gpd.read_file('23.4개시도_안전비상벨현황.geojson')
others_cctv = pd.read_csv('24.4개시도_CCTV설치현황.csv')
suwon_building = pd.read_csv('26.수원시_건축물대장(표제부).csv')
others_park = pd.read_csv('27.4개시도_공원현황.csv')
others_police = pd.read_csv('39.4개시도_경찰관서현황.csv')
others_playground = pd.read_csv('40.4개시도_어린이놀이시설현황.csv')
others_land_use_plan = gpd.read_file('41.4개시도_토지이용계획도.geojson')
others_convenience = gpd.read_file('42.4개시도_편의점현황.geojson')

### 치안센터 기준의 데이터프레임 생성

In [3]:
others_police_box_area.columns=['관할치안센터','관할경찰서','geometry']
others_police_box_area=others_police_box_area.drop_duplicates(['관할치안센터'])
others_police_box_area.reset_index(drop=True,inplace=True)

In [4]:
police_box_4sido = others_police_box_area.copy()
police_box_4sido = police_box_4sido[['관할치안센터','관할경찰서']]
police_box_4sido.sort_values(by=["관할치안센터"], inplace=True)
police_box_4sido.reset_index(inplace = True,drop=True)

### mapping & count 함수 생성

In [5]:
def point_police_box(data_df,oters_police_box_area,col):
    df = data_df.copy()

    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    df['geometry'] = df.apply(lambda x : Point([x['lon'], x['lat']]), axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry', crs = epsg4326)
    
    police_box_result = gpd.sjoin(others_police_box_area, df, how='inner', op="intersects").drop(["index_right"],axis=1)
    
    police_box_result_groupby = police_box_result.groupby("관할치안센터").agg({col:"count"}).reset_index().rename(columns={col:"cnt"})
    
    return police_box_result,police_box_result_groupby

# 카운트용 함수

def point_police_box_count(data_df,others_police_box,col):
    df = data_df.copy()

    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    df['geometry'] = df.apply(lambda x : Point([x['lon'], x['lat']]), axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry', crs = epsg4326)
    
    police_box_result = gpd.sjoin(others_police_box_area, df, how='inner', op="intersects").drop(["index_right"],axis=1)
    
    police_box_result_groupby = police_box_result.groupby("관할치안센터").agg({col:"sum"}).reset_index().rename(columns={col:"cnt"})
    
    return police_box_result,police_box_result_groupby    
def police_box_4sido_count(df,name):
    for i in range(len(df)):
        for j in range(len(police_box_4sido)):
            if df.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터']:
                    police_box_4sido.loc[j, name]=df.loc[i, 'cnt']

#### 지역명 매핑

In [6]:
area_list = []
for i in range(len(police_box_4sido)):
    area_list.append(police_box_4sido['관할경찰서'][i][0:2])
for i in range(len(area_list)):
    if (area_list[i]=='마산') | (area_list[i] == '진해'):
        area_list[i] = '창원'
police_box_4sido['지역'] = area_list


## 27.4개시도_공원현황 mapping

In [7]:
others_park_result, others_park_groupby=point_police_box(others_park,others_police_box_area,'park_gbn')
others_park_result=others_park_result.groupby(['관할치안센터', 'park_gbn']).count()
others_park_result.reset_index(inplace=True)
others_park_result=others_park_result[['관할치안센터','park_gbn','park_nm']]
police_box_4sido_count(others_park_groupby,'공원')

In [8]:
## 공원 분류(안넣어도 됨)
# police_box_4sido['어린이공원']=0
# police_box_4sido['근린공원']=0
# police_box_4sido['소공원']=0
# police_box_4sido['문화공원']=0
# police_box_4sido['역사공원']=0
# police_box_4sido['체육공원']=0
# for i in range(len(others_park_result)):
#     for j in range(len(police_box_4sido)):
#         if others_park_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_park_result.loc[i, 'park_gbn'] == '근린공원':
#             police_box_4sido.loc[j, '근린공원']=others_park_result.loc[i, 'park_nm']
#         elif others_park_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_park_result.loc[i, 'park_gbn'] == '소공원':
#             police_box_4sido.loc[j, '소공원']=others_park_result.loc[i, 'park_nm']
#         elif others_park_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_park_result.loc[i, 'park_gbn'] == '어린이공원':
#             police_box_4sido.loc[j, '어린이공원']=others_park_result.loc[i, 'park_nm']
#         elif others_park_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_park_result.loc[i, 'park_gbn'] == '문화공원':
#             police_box_4sido.loc[j, '문화공원']=others_park_result.loc[i, 'park_nm']
#         elif others_park_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_park_result.loc[i, 'park_gbn'] == '역사공원':
#             police_box_4sido.loc[j, '역사공원']=others_park_result.loc[i, 'park_nm']
#         elif others_park_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_park_result.loc[i, 'park_gbn'] == '체육공원':
#             police_box_4sido.loc[j, '체육공원']=others_park_result.loc[i, 'park_nm']


### 40.4개시도_어린이놀이시설현황/40.4개시도_어린이놀이시설현황_구별종류별

In [9]:
others_playground_result, others_playground_groupby=point_police_box(others_playground,others_police_box_area,'fac_loc')
police_box_4sido_count(others_playground_groupby,'놀이시설')

### 42.4개시도_편의점현황/42.4개시도_편의점현황_업체별수

In [10]:
others_convenience_result, others_convenience_groupby=point_police_box(others_convenience,others_police_box_area,'KSIC_NM')
others_convenience_result=others_convenience_result.groupby(['관할치안센터', 'KSIC_NM']).count()
others_convenience_result.reset_index(inplace=True)
others_convenience_result=others_convenience_result[['관할치안센터','KSIC_NM','BUSINESS_NM']]
police_box_4sido_count(others_convenience_groupby,'편의점')

## 22.4개시도_보안등현황 mapping

In [11]:
others_security_lamp_result, others_security_lamp_groupby=point_police_box_count(others_security_lamp,others_police_box_area,'securitylight_cnt')
police_box_4sido_count(others_security_lamp_groupby,'보안등')

## 23.4개시도_안전비상벨현황 mapping

In [12]:
others_security_bell_result = gpd.sjoin(others_police_box_area, others_security_bell, how='inner', op="intersects").drop(["index_right"],axis=1)
others_security_bell_groupby = others_security_bell_result.groupby("관할치안센터").agg({'bell_us':"count"}).reset_index().rename(columns={'bell_us':"cnt"})
others_security_bell_result=others_security_bell_result.groupby(['관할치안센터', 'bell_us'])['관할경찰서'].count()
others_security_bell_result=others_security_bell_result.to_frame().reset_index().rename(columns={'관할경찰서':"cnt"})
police_box_4sido_count(others_security_bell_groupby,'비상벨')

In [13]:
## 비상등 분류(안넣어도됨)
# police_box_4sido['방범용비상벨']=0
# police_box_4sido['약자보호비상벨']=0
# police_box_4sido['기타비상벨']=0

# for i in range(len(others_security_bell_result)):
#     for j in range(len(police_box_4sido)):
#         if others_security_bell_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_security_bell_result.loc[i, 'bell_us'] == '방범용':
#             police_box_4sido.loc[j, '방범용비상벨']=others_security_bell_result.loc[i, 'cnt']
#         elif others_security_bell_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_security_bell_result.loc[i, 'bell_us'] == '약자보호':
#             police_box_4sido.loc[j, '약자보호비상벨']=others_security_bell_result.loc[i, 'cnt']
#         elif others_security_bell_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_security_bell_result.loc[i, 'bell_us'] == '기타':
#             police_box_4sido.loc[j, '기타비상벨']=others_security_bell_result.loc[i, 'cnt']


## 24.4개시도_CCTV설치현황 mapping

In [14]:
others_cctv_result,others_cctv_groupby = point_police_box_count(others_cctv,others_police_box_area,'cctv_cnt')
others_cctv_result = others_cctv_result.groupby(['관할치안센터','purpose'])['cctv_cnt'].sum()
others_cctv_result=others_cctv_result.to_frame().reset_index()
police_box_4sido_count(others_cctv_groupby,'cctv')

In [15]:
## cctv 분류(안넣어도됨)
# police_box_4sido['생활방범cctv']=0
# police_box_4sido['다목적cctv']=0
# police_box_4sido['어린이보호cctv']=0
# police_box_4sido['교통단속cctv']=0
# police_box_4sido['쓰레기단속cctv']=0
# police_box_4sido['기타cctv']=0
# police_box_4sido['재난재해cctv']=0
# police_box_4sido['시설물관리cctv']=0
# police_box_4sido['차량방법cctv']=0
# police_box_4sido['교통정보수집cctv']=0


# for i in range(len(others_cctv_result)):
#     for j in range(len(police_box_4sido)):
#         if others_cctv_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_cctv_result.loc[i, 'purpose'] == '생활방범':
#             police_box_4sido.loc[j, '생활방범cctv']=others_cctv_result.loc[i, 'cctv_cnt']
#         elif others_cctv_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_cctv_result.loc[i, 'purpose'] == '다목적':
#             police_box_4sido.loc[j, '다목적cctv']=others_cctv_result.loc[i, 'cctv_cnt']
#         elif others_cctv_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_cctv_result.loc[i, 'purpose'] == '어린이보호':
#             police_box_4sido.loc[j, '어린이보호cctv']=others_cctv_result.loc[i, 'cctv_cnt']
#         elif others_cctv_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_cctv_result.loc[i, 'purpose'] == '교통단속':
#             police_box_4sido.loc[j, '교통단속cctv']=others_cctv_result.loc[i, 'cctv_cnt']
#         elif others_cctv_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_cctv_result.loc[i, 'purpose'] == '쓰레기단속':
#             police_box_4sido.loc[j, '쓰레기단속cctv']=others_cctv_result.loc[i, 'cctv_cnt']
#         elif others_cctv_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_cctv_result.loc[i, 'purpose'] == '기타':
#             police_box_4sido.loc[j, '기타cctv']=others_cctv_result.loc[i, 'cctv_cnt']
#         elif others_cctv_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_cctv_result.loc[i, 'purpose'] == '재난재해':
#             police_box_4sido.loc[j, '재난재해cctv']=others_cctv_result.loc[i, 'cctv_cnt']
#         elif others_cctv_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_cctv_result.loc[i, 'purpose'] == '시설물관리':
#             police_box_4sido.loc[j, '시설물관리cctv']=others_cctv_result.loc[i, 'cctv_cnt']
#         elif others_cctv_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_cctv_result.loc[i, 'purpose'] == '차량방법':
#             police_box_4sido.loc[j, '차량방법cctv']=others_cctv_result.loc[i, 'cctv_cnt']
#         elif others_cctv_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_cctv_result.loc[i, 'purpose'] == '교통정보수집':
#             police_box_4sido.loc[j, '교통정보수집cctv']=others_cctv_result.loc[i, 'cctv_cnt']

## 112 신고이력 관련 데이터 가공
- 20개 이상의 사건종별 데이터를 6개의 중분류로 매핑해주기
- 교통 / 질서유지 / 중요범죄 / 기타범죄 / 기타_경찰업무 / 타기관_기타

## 112 데이터 전처리 함수

In [16]:
def 사건종별_분류(x):
    교통 = ["교통사고","교통불편","교통위반"]
    질서유지 = ["기타경범","주취자","행패소란","보호조치","시비","위험방지","청소년비행","무전취식승차"]
    중요범죄 = ["절도","가정폭력","폭력"]
    기타범죄 = ["기타형사범","도박"]
    기타_경찰업무 = ["상담문의","분실습득","비상벨","실종(실종아동 등)"]
    타기관_기타 = ["기타_타기관","소음","서비스요청"]
    
    if x in 교통:
        return "교통"
    if x in 질서유지:
        return "질서유지"
    if x in 중요범죄:
        return "중요범죄"
    if x in 기타범죄:
        return "기타범죄"
    if x in 기타_경찰업무:
        return "기타_경찰업무"
    if x in 타기관_기타:
        return "타기관_기타"

In [17]:
def time_cut(x):
    if 6 > x >= 2:
        return '새벽'
    elif 11 > x >= 6:
        return '오전'
    elif 15 > x >= 11:
        return '점심'
    elif 18 > x >= 15:
        return '오후'
    elif 22 > x >= 18:
        return '저녁'
    else:
        return '심야'

In [18]:
others_112.columns = ["사건종별","접수완료일자","접수완료시각","도착일시","관할치안센터","관할경찰서"]
suwon_112.columns = ["사건종별","접수완료일자","접수완료시각","도착일시","gid","관할치안센터","관할경찰서"]

In [19]:
def 신고이력데이터_처리(data):
    
    others_112 = data.copy()
    others_112["접수완료시각"] = others_112["접수완료시각"].str.replace("'","")
    others_112["접수완료일자"] = others_112["접수완료일자"].astype(str)
    #others_112["도착일시"] = others_112["도착일시"].astype(int).astype(str)
    # 도착일시가 NaN인 경우 존재 -> 도착일시가 NaN이 아닌 경우에 대해서만 출동 소요시각 구하기

    others_112["접수완료일시"] = others_112["접수완료일자"] + others_112["접수완료시각"]
    others_112["접수완료일시"] = pd.to_datetime(others_112["접수완료일시"],format="%Y%m%d%H%M%S")

    others_112["년"] = others_112["접수완료일시"].apply(lambda x : x.year)
    others_112["월"] = others_112["접수완료일시"].apply(lambda x : x.month)
    others_112["일"] = others_112["접수완료일시"].apply(lambda x : x.day)
    others_112["요일"] = others_112["접수완료일시"].apply(lambda x : x.weekday())
    others_112["시"] = others_112["접수완료일시"].apply(lambda x : x.hour)
    others_112["분"] = others_112["접수완료일시"].apply(lambda x : x.minute)
    others_112["초"] = others_112["접수완료일시"].apply(lambda x : x.second)

    # 도착일시가 정확히 기록된 경우와 그렇지 않은 경우로 나누기

    def get_도착일시(x):
        if len(str(x)) < 14:
            return np.NaN
        else:
            return str(int(float(x)))

    others_112["도착일시"] = others_112["도착일시"].apply(lambda x : get_도착일시(x))
    others_112["도착일시"] = pd.to_datetime(others_112["도착일시"],format="%Y%m%d%H%M%S")

    # 출동하는데 걸린 소요 시간
    others_112["소요시간"] = others_112["도착일시"] - others_112["접수완료일시"]

    # 사건종별 중분류
    others_112["사건종별_중분류"] = others_112["사건종별"].apply(lambda x : 사건종별_분류(x))
    
    # 시간대별 분류
    others_112["시간대"] = others_112["시"].apply(lambda x : time_cut(x))
    
    return others_112

In [ ]:
suwon_112_get = 신고이력데이터_처리(suwon_112)
others_112_get = 신고이력데이터_처리(others_112)

suwon_112_groupby = suwon_112_get.groupby("관할치안센터").agg({'gid':"count"}).reset_index().rename(columns={'gid':"cnt"})
suwon_112_get_groupby = suwon_112_get.groupby(['관할치안센터','사건종별_중분류'])['gid'].count().reset_index().rename(columns={'gid':"cnt"})

others_112_groupby = others_112_get.groupby("관할치안센터").agg({'사건종별':"count"}).reset_index().rename(columns={'사건종별':"cnt"})
others_112_get_groupby = others_112_get.groupby(['관할치안센터','사건종별_중분류'])['사건종별'].count().reset_index().rename(columns={'사건종별':"cnt"})

### 112 신고 데이터 치안센터 영역과 매핑하는 함수

In [ ]:
def police_box_4sido_112_mapping(df1,df2):
    police_box_4sido_count(df1,'전체112')
    for i in range(len(df2)):
        for j in range(len(police_box_4sido)):
            if df2.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and df2.loc[i, '사건종별_중분류'] == '기타_경찰업무':
                police_box_4sido.loc[j, '112_기타_경찰업무']=df2.loc[i, 'cnt']
            elif df2.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and df2.loc[i, '사건종별_중분류'] == '중요범죄':
                police_box_4sido.loc[j, '112_중요범죄']=df2.loc[i, 'cnt']
            elif df2.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and df2.loc[i, '사건종별_중분류'] == '교통':
                police_box_4sido.loc[j, '112_교통']=df2.loc[i, 'cnt']
            elif df2.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and df2.loc[i, '사건종별_중분류'] == '타기관_기타':
                police_box_4sido.loc[j, '112_타기관_기타']=df2.loc[i, 'cnt']
            elif df2.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and df2.loc[i, '사건종별_중분류'] == '질서유지':
                police_box_4sido.loc[j, '112_질서유지']=df2.loc[i, 'cnt']
            elif df2.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and df2.loc[i, '사건종별_중분류'] == '기타범죄':
                police_box_4sido.loc[j, '112_기타범죄']=df2.loc[i, 'cnt']


In [ ]:
police_box_4sido['전체112']=0
police_box_4sido['112_기타_경찰업무']=0
police_box_4sido['112_중요범죄']=0
police_box_4sido['112_교통']=0
police_box_4sido['112_타기관_기타']=0
police_box_4sido['112_질서유지']=0
police_box_4sido['112_기타범죄']=0

In [ ]:
police_box_4sido_112_mapping(suwon_112_groupby,suwon_112_get_groupby)

In [ ]:
police_box_4sido_112_mapping(others_112_groupby,others_112_get_groupby)

4개시도_유흥업소현황 mapping

In [ ]:
others_alcohol = gpd.read_file('8.4개시도_유흥업소현황.geojson')[['BUSINESS_NM','COM_LC_NM','COM_MC_NM',
                'COM_SC_NM','KSIC_NM','SGG_NM',
       'ADB_EMD_NM','EMD_NM','ROAD_NM','geometry']]
others_alcohol.columns = ["상호명","상권업종대분류명","상권업종중분류명","상권업종소분류명","표준산업분류명",
                         "시군구명","행정동명","법정동명","도로명","geometry"]

In [ ]:
others_alcohol_result = gpd.sjoin(others_police_box_area, others_alcohol, how='inner', op="intersects").drop(["index_right"],axis=1)
others_alcohol_result=others_alcohol_result.groupby(['관할치안센터'])['관할경찰서'].count().to_frame().reset_index().rename(columns={'관할경찰서':"cnt"})
police_box_4sido_count(others_alcohol_result,'유흥업소')